# Talking Data

The purpose of this project is to use mobile user data to predict the age group of the user.  The age groups are split betweeen either female or male, then from below 23,to above 43.  Here is what it looks for females: F23-, F24-26, F27-28, F29-32, F33-42, F43+. 

The data is from the Chinese company TalkingData. There are several data sets, each a unique csv: gender_age_train, gender_age_test, events, app_events, app_labels, label_categories and phone_brand_device_model.  

In [177]:
#Importing the packages needed for this analysis
from pygeocoder import Geocoder
import pandas as pd
import io
import numpy as np
import pickle
import math
import re
import reverse_geocoder as rg
from datetime import *
from scipy import stats
import dateutil.parser
from IPython.display import display
from sklearn import preprocessing
from googletrans import Translator
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, RobustScaler
from IPython.core.interactiveshell import InteractiveShell
import goslate
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
%config InlineBackend.figure_format = 'png'

plt.rcParams['figure.dpi']= 400

In [178]:
#Connecting to SQL database on an AWS EC2 instance
from sqlalchemy import create_engine
import pandas as pdb
cnx = create_engine('postgresql://djmor@34.234.69.231/djmor')

In [179]:
#Loading in coordinates to fast track reverse geocoding
geo = rg.RGeocoder(mode=2, verbose=True, stream=io.StringIO(open('rg_cities1000.csv', encoding='utf-8').read()))

In [204]:
#List of classification types 
classes = {'F23-':0, 'F24-26':1,'F27-28':2,'F29-32':3, 'F33-42':4, 'F43+':5, 'M22-':6, 'M23-26':7, 'M27-28':8, 'M29-31':9, 'M32-38':10, 'M39+':11}
           
           

In [181]:
#Creating all the data frames
device_brand = pd.read_csv('phone_brand_device_model.csv')
applabeldf = pd.read_csv('app_labels.csv')
labelcat = pd.read_csv('label_categories.csv')
genderdf = pd.read_csv('gender_age_train.csv')
eventsdf = pd.read_csv('~/Documents/events.csv')
appevents = pd.read_csv('~/Documents/app_events.csv')
testdf = pd.read_csv('gender_age_test.csv')

In [115]:
testdf.head()
genderdf.head()

,device_id
0,1002079943728939269
1,-1547860181818787117
2,7374582448058474277
3,-6220210354783429585
4,-5893464122623104785


,device_id,gender,age,group
0,-8076087639492063270,M,35,M32-38
1,-2897161552818060146,M,35,M32-38
2,-8260683887967679142,M,35,M32-38
3,-4938849341048082022,M,30,M29-31
4,245133531816851882,M,30,M29-31


In [116]:
#Getting an idea on how each df connects with each other
device_brand.head()
appevents.head()
labelcat.head()
eventsdf.head()
applabeldf.head()

,device_id,phone_brand,device_model
0,-8890648629457979026,小米,红米
1,1277779817574759137,小米,MI 2
2,5137427614288105724,三星,Galaxy S4
3,3669464369358936369,SUGAR,时尚手机
4,-5019277647504317457,三星,Galaxy Note 2


,event_id,app_id,is_installed,is_active
0,2,5927333115845830913,1,1
1,2,-5720078949152207372,1,0
2,2,-1633887856876571208,1,0
3,2,-653184325010919369,1,1
4,2,8693964245073640147,1,1


,label_id,category
0,1,NaN
1,2,game-game type
2,3,game-Game themes
3,4,game-Art Style
4,5,game-Leisure time


,event_id,device_id,timestamp,longitude,latitude
0,1,29182687948017175,2016-05-01 00:55:25,121.38,31.24
1,2,-6401643145415154744,2016-05-01 00:54:12,103.65,30.97
2,3,-4833982096941402721,2016-05-01 00:08:05,106.60,29.70
3,4,-6815121365017318426,2016-05-01 00:06:40,104.27,23.28
4,5,-5373797595892518570,2016-05-01 00:07:18,115.88,28.66


,app_id,label_id
0,7324884708820027918,251
1,-4494216993218550286,251
2,6058196446775239644,406
3,6058196446775239644,407
4,8694625920731541625,406


In [118]:
#Translating Phone brand to English for easier comprehension
translate = {'三星':'samsung','天语':'Ktouch','海信':'hisense','联想':'lenovo','欧比':'obi','爱派尔':'ipair','努比亚':'nubia',\
             '优米':'youmi','朵唯':'dowe','黑米':'heymi','锤子':'hammer','酷比魔方':'koobee','美图':'meitu','尼比鲁':'nibilu','一加':'oneplus','优购':'yougo',\
             '诺基亚':'nokia','金立':'jinli','中兴':'zhongxing','糖葫芦':'candy','中国移动':'ccmc','语信':'yuxin','基伍':'kiwu','青橙':'greeno','华硕':'asus',\
             '夏新':'panosonic','维图':'weitu','艾优尼':'aiyouni','摩托罗拉':'moto','乡米':'xiangmi','米奇':'micky','大可乐':'bigcola',\
             '沃普丰':'wpf','神舟':'hasse','摩乐':'mole','酷派':'kupai','飞秒':'fs','米歌':'mige','富可视':'fks','德赛':'desci','梦米':'mengmi','乐视':'lshi','小杨树':'smallt','纽曼':'newman','邦华':'banghua',\
             'E派':'epai','华为':'Huawei','小米':'xiaomi','魅族':'meizu', '易派':'epai','普耐尔':'pner','欧新':'ouxin','西米':'ximi','海尔':'haier','波导':'bodao','糯米':'nuomi','唯米':'weimi','酷珀':'kupo','谷歌':'google','昂达':'ada','聆韵': 'lingyun'}

In [119]:
#Translating phone brand from Chinese to English

for index, i in enumerate(device_brand['phone_brand']):
    if i in translate:
        device_brand.loc[index,'phone_brand'] = translate[i]


In [120]:
#Using an offline reverse geocoder to get city and country data, also this takes a long time to run...
counter =0
eventsdf['city'] = ''
tup_ls = []
for i,j in zip(eventsdf['latitude'],eventsdf['longitude']):
        lalong = (i,j)
        tup_ls.append(lalong)




In [121]:
q_list = geo.query(tup_ls)
name_l = []
cc_l = []
for q in q_list:
    name_l.append(q['name'])
    cc_l.append(q['cc'])

In [122]:
eventsdf['city'] = name_l
eventsdf['cc'] = cc_l

In [123]:
eventsdf

,event_id,device_id,timestamp,longitude,latitude,city,cc
0,1,29182687948017175,2016-05-01 00:55:25,121.38,31.24,Changzheng,CN
1,2,-6401643145415154744,2016-05-01 00:54:12,103.65,30.97,Zitong,CN
2,3,-4833982096941402721,2016-05-01 00:08:05,106.60,29.70,Huixing,CN
3,4,-6815121365017318426,2016-05-01 00:06:40,104.27,23.28,Wenshan City,CN
4,5,-5373797595892518570,2016-05-01 00:07:18,115.88,28.66,Taohua,CN
5,6,1476664663289716375,2016-05-01 00:27:21,0.00,0.00,Takoradi,GH
6,7,5990807147117726237,2016-05-01 00:15:13,113.73,23.00,Nancheng,CN
7,8,1782450055857303792,2016-05-01 00:15:35,113.94,34.70,Zhengzhou,CN
8,9,-2073340001552902943,2016-05-01 00:15:33,0.00,0.00,Takoradi,GH
9,10,-8195816569128397698,2016-05-01 00:41:31,119.34,26.04,Gaishan,CN


In [126]:
#Filling NaN with none for better processing
labelcat.fillna('None', inplace = True)

In [135]:
#Pulling out Month data
eventsdf['time_dl'] = eventsdf['timestamp'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d  %H:%M:%S').time())
eventsdf['wkday'] = eventsdf['timestamp'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d  %H:%M:%S').weekday())

In [ ]:
eventsdf['class'] = eventsdf['timestamp'].apply(lambda x: classes[x])

In [146]:
#Writing a pickle file to save data

with open('eventsdf.pkl', 'wb') as picklefile:
    pickle.dump(eventsdf, picklefile)

In [140]:
eventsdf.head()

,event_id,device_id,timestamp,longitude,latitude,city,cc,time_dl,wkday
0,1,29182687948017175,2016-05-01 00:55:25,121.38,31.24,Changzheng,CN,00:55:25,6
1,2,-6401643145415154744,2016-05-01 00:54:12,103.65,30.97,Zitong,CN,00:54:12,6
2,3,-4833982096941402721,2016-05-01 00:08:05,106.60,29.70,Huixing,CN,00:08:05,6
3,4,-6815121365017318426,2016-05-01 00:06:40,104.27,23.28,Wenshan City,CN,00:06:40,6
4,5,-5373797595892518570,2016-05-01 00:07:18,115.88,28.66,Taohua,CN,00:07:18,6


## Merging the Data

This section may become obsolete after I get the SQL database going, but as for now, it exists.***Delete Eventually***

I will be using Inner joins to merge the data frames.  My guess is that both training and test data is in each of the auxilary columns, so I only want the intersection of the training dataframe and the auxilary dataframes.

In this section, I also added some extra data.  I pulled out the datetime string and converted into into a readable datetime object.  The time of day and day of the week may not make much of a difference.  It depends on the culture of the society.

In [254]:
#Loading DF into jupyter notebook
eventsdf = pd.read_pickle('eventsdf.pkl')

In [255]:
def supermerge(dfleft):
    result = pd.merge(dfleft, device_brand, how = 'inner', on = 'device_id')
    result = pd.merge(result, eventsdf, how = 'inner', on = 'device_id')
    result = pd.merge(result, appevents, how = 'inner', on = 'event_id')
    result = pd.merge(result, applabeldf, how = 'inner', on = 'app_id')
    result = pd.merge(result, labelcat, how = 'inner', on = 'label_id')
    return result

In [256]:
genderdf['class'] = genderdf['group'].apply(lambda x: classes[x])

In [257]:
traindf = supermerge(genderdf)

In [258]:
traindf.drop(['gender','age', 'longitude', 'latitude','label_id', 'timestamp', 'is_installed','device_id'], axis=1, inplace = True)

In [196]:
len(traindf.event_id.unique())

556375

In [198]:
len(traindf.category.unique())

451

## Preprocessing

The data has many categorical variables that we are using, so a little preprocessing is needed.  I will be using OneHotEncoder to do this for me.

In [261]:
from sklearn import preprocessing
enc = preprocessing.OneHotEncoder()

In [ ]:
newX = enc.fit(X)

In [ ]:
newX

## Logistic Classification Analysis

The amount of data in the dataframe will make most of the analysis long.  There are 79 million observations, and even splitting it up will make it hard.  Some of the data is in different countries, so dividing it into subsections such as Country, City and Device Brand may make it easier for analysis.  

In [220]:
# STEP 1: split X and y into training and testing sets
y = traindf['class']
X= traindf.drop(['group','class'], axis=1)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [221]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
logit = LogisticRegression()

In [244]:
logit.fit(X_train, y_train)

ValueError: could not convert string to float: 'Total Cost 1'

## Looking at Trees

Below is looking at the Decision Tree and Random Forests classification methods to see how it differs from the analysis above.



In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier